# ResNetの学習
- MNISTの学習と乃木坂画像の学習がしたい

In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
sys.path.append('../misc')
sys.path.append('/Users/Daigo/gitclone/keras-resnet')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import h5py
import pickle

from resnet import ResnetBuilder
from Nogi import NogiFaceImport
from MNISTutil import loadMNIST

Using TensorFlow backend.


In [2]:
nogi = NogiFaceImport(path='/Users/Daigo/Data/NogiFace/')

loading Nogizaka46 Face Data ...


In [3]:
data_mnist = loadMNIST()
data_nogi = nogi.load(front_only=True, 
                      image_size=(128, 128))

data shape : (N, 1, 28, 28)
front only : True
H/W size : (128, 128)
grayscale : False
one hot : True
train data shape : (4896, 128, 128, 3), train label shape : (4896, 5)
test data shape : (544, 128, 128, 3), test label shape : (544, 5)


In [21]:
# handle Nogizaka46 data
(x_train, t_train), (x_test, t_test) = data_nogi

In [22]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

In [6]:
batch_size = 128
nb_classes = 5
nb_epoch = 10

In [24]:
model = ResnetBuilder.build_resnet_18(input_shape=(3, 128, 128), num_outputs=5)

/Users/Daigo/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer="he_normal", kernel_regularizer=<keras.reg..., padding="same", kernel_size=(7, 7), strides=(2, 2), filters=64)`
  '` call to the Keras 2 API: ' + signature)
/Users/Daigo/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(padding="same", pool_size=(3, 3), strides=(2, 2))`
  '` call to the Keras 2 API: ' + signature)
/Users/Daigo/.pyenv/versions/anaconda3-4.1.1/lib/python3.5/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_initializer="he_normal", kernel_regularizer=<keras.reg..., padding="same", kernel_size=(3, 3), strides=(1, 1), filters=64)`
  '` call to the Keras 2 API: ' + signature)
/Users/Daigo/gitclone/keras

In [25]:
model.compile(loss = 'categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [26]:
history = model.fit(x_train, t_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.1)

Train on 4406 samples, validate on 490 samples
Epoch 1/10
 256/4406 [>.............................] - ETA: 877s - loss: 2.2696 - acc: 0.3359 

KeyboardInterrupt: 